# Graphical Lasso Demonstration
This notebook performs the Graphical Lasso on a 4‑dimensional Gaussian random vector whose precision matrix corresponds to the $4$-dimensional graph with two cliques $\{1,2,3\}$ and $\{2 , 3 , 4\}$. The estimation is performed using:

1. **`graphical_lasso()`** from `sklearn.covariance`
2. A **custom implementation** solved using the **BFGS algorithm** from `scipy.optimize.minimize`

## 1. Define the graph and precision matrix

We first define the precision matrix 
$$
\Theta =
\begin{pmatrix}
10 & 5 & 3 & 0 \\
5 & 10 & 5 & 3 \\
3 & 5 & 10 & 5 \\
0 & 3 & 5 & 10
\end{pmatrix}.
$$

We define the inverse $\Sigma= \Theta^{-1}$.

In [1]:
import numpy as np
import pandas as pd

d = 4
Theta = np.array([
    [10,  5,  3, 0],
    [ 5, 10,  5,  3],
    [ 3,  5, 10 ,   5],
    [ 0,  3,  5, 10]
])
Sigma = np.linalg.inv(Theta)

## 2. Simulate Gaussian sample

We simulate form a centered Gaussian distribution with covariance matrix $\Sigma$

In [2]:
import random 
random.seed(7)
N = 10**4
mean = np.zeros(d)
X = np.random.multivariate_normal(mean, Sigma, size=N)
X[:5]

array([[ 0.0249882 ,  0.40326314, -0.33943285, -0.00079558],
       [-0.32807514,  0.66074409, -0.15315525, -0.14403168],
       [-0.49390619,  0.07857231, -0.2781498 ,  0.543012  ],
       [ 0.23711111, -0.01183191, -0.05047926, -0.37161609],
       [ 0.36719285, -0.06415889, -0.00331654, -0.16394107]])

In [4]:
EstimSigma = np.cov(X, rowvar=False)
print("This is the sample covariance matrix computed from the sample:\n", EstimSigma)

This is the sample covariance matrix computed from the sample:
 [[ 0.14063144 -0.06848973 -0.02245653  0.02948985]
 [-0.06848973  0.16717006 -0.05238776 -0.02333097]
 [-0.02245653 -0.05238776  0.16912751 -0.0706653 ]
 [ 0.02948985 -0.02333097 -0.0706653   0.14138246]]


## 3. Graphical Lasso Using `sklearn.covariance.graphical_lasso`
The `graphical_lasso()` function solves the penalized likelihood problem
$$
argmax_\Theta \; \log\det(\Theta) - \operatorname{tr}(S\Theta) - \lambda \|\Theta\|_1
$$

where:
- **S** is the sample covariance matrix
- **λ** controls the sparsity of the solution
- The algorithm used is the **GLasso algorithm** from Friedman et al. (2008, *Biostatistics*)


As a penalization term, we fix $\lambda = 10^{-4}$. 

In [5]:
from sklearn.covariance import  graphical_lasso

covariance , precision = graphical_lasso(EstimSigma , alpha=   pow(10 , -4))

print("Estimated Precision Matrix (Theta) using the glasso package:\n", precision)

Estimated Precision Matrix (Theta) using the glasso package:
 [[ 9.99241135  5.04961626  2.97528855  0.22335392]
 [ 5.04961626 10.07758961  5.09381078  3.14134341]
 [ 2.97528855  5.09381078 10.06653888  5.23853998]
 [ 0.22335392  3.14134341  5.23853998 10.15703503]]


## 4. Graphical Lasso using an implementation of the LASSO-loglikelihood function 
We implement the Lasso log-Likelihood function given by 

$$
L(\Theta) =  \log \det(\Theta) - trace (S \cdot \Theta) -\lambda \| \Theta \|_1,
$$
where $\| \Theta \|_1$ is the sum of the absolute value of the entries from the precision matrix $\Theta$ and $S$ is MLE covariance matrix, that is:
$$
S = \frac{1}{n} \sum_{i =1}^n X_i^t X_i
$$

We minimize the Lasso-loglikelihood function using the function **minimize()** with the **Broyden–Fletcher–Goldfarb–Shanno algorithm** method and we randomly choose initial values for the first step of the minimization. We get the estimate $\hat{\Theta}_2$.

In [6]:
from scipy.optimize import minimize
### Helper: Build a Symmetric Matrix from a Vector
def construct_symmetric_matrix(theta_vec):
    Theta_mat = np.zeros((d, d))
    idx = np.triu_indices(d)
    Theta_mat[idx] = theta_vec
    Theta_mat = Theta_mat + Theta_mat.T - np.diag(np.diag(Theta_mat))
    return Theta_mat

### Penalized Log-Likelihood
def penalized_log_likelihood(theta_vec, S , lamb):
    Theta_mat = construct_symmetric_matrix(theta_vec)
    det = np.linalg.det(Theta_mat)
    if det <= 0:
        return -np.inf
    sum_Theta = np.absolute(Theta_mat).sum()    
    return np.log(det) - np.trace(S @ Theta_mat )  - lamb * sum_Theta + lamb * np.absolute(np.diag(Theta_mat)).sum()


### Initial Matrix
A = np.random.randn(d, d)
Theta0 = A @ A.T + d * np.eye(d)
theta0_vec = Theta0[np.triu_indices(d)]


lamb = pow(10 , -4)

### Optimization via BFGS

res = minimize(lambda t: -penalized_log_likelihood(t, EstimSigma , lamb),
               theta0_vec, 
               method = "BFGS"
              )

Theta_MLE = construct_symmetric_matrix(res.x)
print("Estimated Precision Matrix (Theta) using implemented LASSO log-likelihood function:\n" , Theta_MLE)  


Estimated Precision Matrix (Theta) using implemented LASSO log-likelihood function:
 [[ 9.99154076  5.04895117  2.97518607  0.22361391]
 [ 5.04895117 10.07699086  5.09371132  3.14156692]
 [ 2.97518607  5.09371132 10.06682585  5.23895107]
 [ 0.22361391  3.14156692  5.23895107 10.1575178 ]]


Note that both methods give the same results. Both estimations give good estimation for the precision matrix $\Theta$, howveer both of them fail to detect $\Theta_{14}=0$. One reason could be a bad choice for the penalization parameter $\lambda$. There exists some known method to choose this penalization parameter: cross validation, AIC, BIC, etc...